In [1]:
# imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import struct
from array import array
from os.path  import join
import random


# started from this video: https://www.youtube.com/watch?v=w8yWXqWQYmU&t=579s

In [2]:
#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath

    def read_images_labels(self, images_filepath, labels_filepath):
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())

        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img

        return images, labels

    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)

In [3]:
#
# Verify Reading Dataset via MnistDataloader class
#
%matplotlib inline

#
# Set file paths based on added MNIST Datasets
#
input_path = './MNIST_ORG/'
training_images_filepath = join(input_path, 'train-images.idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels.idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images.idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels.idx1-ubyte')

#
# Helper function to show a list of images with their relating titles
#
def show_images(images, title_texts):
    cols = 5
    rows = int(len(images)/cols) + 1
    plt.figure(figsize=(30,20))
    index = 1
    for x in zip(images, title_texts):
        image = x[0]
        title_text = x[1]
        plt.subplot(rows, cols, index)
        plt.imshow(image, cmap=plt.cm.gray)
        if (title_text != ''):
            plt.title(title_text, fontsize = 15);
        index += 1

#
# Load MINST dataset
#
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

# x_train and x_test are the images in a list of 28x28 arrays
# y_train and y_test are the number it should represent as a list of integers
# training data is 60000 labelled images, test is 10000 images

# need to reshape the 28x28 arrays into one 784x1 array, since the input to the matrix multiplciation has to be a 1xn matrix, not nxn
x_train = np.array(x_train).reshape(60000,784).T / 255
x_test = np.array(x_test).reshape(10000,784).T / 255
y_train = np.array(y_train).reshape(60000)
y_test = np.array(y_test).reshape(10000)

# x_train = x_train[,0:10000]
# y_train = y_train[0:10000,]

#
# Show some random training and test images
#
# images_2_show = []
# titles_2_show = []
# for i in range(0, 10):
#     r = random.randint(1, 60000)
#     images_2_show.append(x_train[r])
#     titles_2_show.append('training image [' + str(r) + '] = ' + str(y_train[r]))
#
# for i in range(0, 5):
#     r = random.randint(1, 10000)
#     images_2_show.append(x_test[r])
#     titles_2_show.append('test image [' + str(r) + '] = ' + str(y_test[r]))
#
# show_images(images_2_show, titles_2_show)

In [4]:

# print(len(x_train))
# print(len(y_train))
# print(len(x_test))
# print(len(y_test))
#
# print(len(x_train[0]))
# print(y_train[0])
# print(len(x_test[0][0]))
# print(y_test[0])

print(x_train.shape) # confirmed right shape as in video
print(y_train.shape)
print(y_train[0:20])

(784, 60000)
(60000,)
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]


In [5]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [49]:
def init_params():
    W1 = (np.random.rand(10, 784) - 0.5)*np.sqrt(2/784)
    b1 = (np.random.rand(10, 1) - 0.5)/100
    W2 = (np.random.rand(10, 10) - 0.5)*np.sqrt(1/10)
    b2 = (np.random.rand(10, 1) - 0.5)/100
    # print(W1,b1, W2, b2)
    return W1, b1, W2, b2

def ReLU(Z):
    # print("Before ReLU: ",Z[0:10,0:10])
    return np.maximum(0,Z)

def softMax(Z):
    L = Z - np.max(Z, axis=0, keepdims=True)
    expL = np.exp(L)
    return expL / expL.sum(axis=0, keepdims=True)
    # # print(Z[0:10,0:10])
    # sum = np.sum(np.exp(Z))
    # # print("Z denominator: ", sum)
    # return np.exp(Z)/sum

def forward_prop(W1, b1, W2, b2, X):
    # print("Forward Prop beginning:")
    # print("X: ",X[0:784,0:10])
    Z1 = W1.dot(X) + b1
    # print("Z1 shape: ", Z1.shape)
    # print(X.shape)
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softMax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def back_prop(Z1, A1, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    # dW1Norm = np.linalg.norm(dW1)
    # W1Norm = np.linalg.norm(W1)
    # print("Norms before updating: W1", W1Norm, ", dW1: ", dW1Norm)
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2

In [50]:

def get_predictions(A):
    return np.argmax(A, 0)

def get_accuracy(predictions, Y):
    # print(predictions[10:30], Y[10:30])
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if (i % 20 == 0):
            print("i: ", i, ", Accuracy: ", "{:.4f}".format(get_accuracy(get_predictions(A2), Y)), ", Z1 denominator: ", "{:.2e}".format(np.sum(np.exp(Z1))))
    return W1, b1, W2, b2

In [51]:
W1, b1, W2, b2 = gradient_descent(x_train, y_train, 500, 0.01)

i:  0 , Accuracy:  0.1212 , Z1 denominator:  6.00e+05
i:  20 , Accuracy:  0.1787 , Z1 denominator:  6.05e+05
i:  40 , Accuracy:  0.2140 , Z1 denominator:  6.24e+05
i:  60 , Accuracy:  0.2147 , Z1 denominator:  6.50e+05
i:  80 , Accuracy:  0.2105 , Z1 denominator:  6.77e+05
i:  100 , Accuracy:  0.2105 , Z1 denominator:  7.06e+05
i:  120 , Accuracy:  0.2125 , Z1 denominator:  7.36e+05
i:  140 , Accuracy:  0.2146 , Z1 denominator:  7.70e+05
i:  160 , Accuracy:  0.2170 , Z1 denominator:  8.08e+05
i:  180 , Accuracy:  0.2209 , Z1 denominator:  8.52e+05
i:  200 , Accuracy:  0.2270 , Z1 denominator:  9.00e+05
i:  220 , Accuracy:  0.2349 , Z1 denominator:  9.54e+05
i:  240 , Accuracy:  0.2452 , Z1 denominator:  1.01e+06
i:  260 , Accuracy:  0.2584 , Z1 denominator:  1.08e+06
i:  280 , Accuracy:  0.2749 , Z1 denominator:  1.16e+06
i:  300 , Accuracy:  0.2963 , Z1 denominator:  1.25e+06
i:  320 , Accuracy:  0.3209 , Z1 denominator:  1.35e+06
i:  340 , Accuracy:  0.3491 , Z1 denominator:  1.47e+0